In [10]:
import tensorflow as tf  # Import TensorFlow library
from tensorflow.keras import layers, models  # Import layers and models from Keras
import numpy as np  # Import NumPy library
import datetime  # Import datetime for timing
import multiprocessing  # Import multiprocessing for parallel processing

def download_mnist_images():
    # Load the MNIST dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    # Reshape and normalize the training and testing images
    x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
    x_test = x_test.reshape(-1, 784).astype('float32') / 255.0
    # Convert labels to categorical (one-hot encoded) format
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test)

def create_part1():
    # Define the first part of the model
    inputs = tf.keras.Input(shape=(784,))  # Input layer with shape 784
    x = layers.Dense(128, activation='relu')(inputs)  # Dense layer with 128 units and ReLU activation
    outputs = layers.Dense(64, activation='relu')(x)  # Dense layer with 64 units and ReLU activation
    model = tf.keras.Model(inputs, outputs)  # Create the model
    return model

def create_part2():
    # Define the second part of the model
    inputs = tf.keras.Input(shape=(64,))  # Input layer with shape 64
    outputs = layers.Dense(10, activation='softmax')(inputs)  # Output layer with 10 units and softmax activation
    model = tf.keras.Model(inputs, outputs)  # Create the model
    return model

def create_combined_model(part1, part2):
    # Combine part1 and part2 into a single model
    inputs = tf.keras.Input(shape=(784,))  # Input layer with shape 784
    x = part1(inputs)  # Pass input through part1
    outputs = part2(x)  # Pass the output of part1 through part2
    model = tf.keras.Model(inputs, outputs)  # Create the combined model
    return model

def evaluate_model(model, x_test, y_test):
    # Evaluate the model on the test dataset
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)  # Evaluate and get the loss and accuracy
    return test_loss, test_acc

def train_part1(x_train, y_train, epochs, batch_size, return_dict, event):
    # Train part1 of the model
    part1 = create_part1()  # Create part1 model
    part1.compile(optimizer='adam', loss='mse')  # Compile the model with MSE loss
    part1.fit(x_train, x_train, batch_size=batch_size, epochs=epochs, verbose=1)  # Train the model
    intermediate_output = part1.predict(x_train)  # Get the intermediate output
    return_dict['part1'] = part1  # Store the trained part1 model in the shared dictionary
    return_dict['intermediate_output'] = intermediate_output  # Store the intermediate output in the shared dictionary
    event.set()  # Signal that part1 has finished training

def train_part2(y_train, epochs, batch_size, return_dict, event):
    # Wait for part1 to finish and the intermediate output to be ready
    event.wait()
    # Train part2 of the model
    intermediate_output = return_dict['intermediate_output']  # Retrieve intermediate output
    part2 = create_part2()  # Create part2 model
    part2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Compile the model with categorical crossentropy loss
    part2.fit(intermediate_output, y_train, batch_size=batch_size, epochs=epochs, verbose=1)  # Train the model
    return_dict['part2'] = part2  # Store the trained part2 model in the shared dictionary

def train_and_evaluate_parallel_parts():
    # Load and preprocess the MNIST dataset
    (x_train, y_train), (x_test, y_test) = download_mnist_images()

    # Training parameters
    epochs = 10
    batch_size = 128

    # Create a manager for shared dictionary and an event for synchronization
    manager = multiprocessing.Manager()
    return_dict = manager.dict()  # Shared dictionary to store results from processes
    event = multiprocessing.Event()  # Event to signal when part1 has finished training

    # Create processes for training part1 and part2
    process1 = multiprocessing.Process(target=train_part1, args=(x_train, y_train, epochs, batch_size, return_dict, event))
    process1.start()  # Start training part1

    # Now start part2 training
    process2 = multiprocessing.Process(target=train_part2, args=(y_train, epochs, batch_size, return_dict, event))
    process2.start()  # Start training part2

    # Wait for both processes to complete
    process1.join()
    process2.join()

    part1 = return_dict['part1']  # Retrieve the trained part1 model
    part2 = return_dict['part2']  # Retrieve the trained part2 model

    # Combine part1 and part2 for evaluation
    combined_model = create_combined_model(part1, part2)

    # Evaluate the combined model on the test dataset
    intermediate_output_test = part1.predict(x_test)  # Get intermediate output for test data
    test_loss, test_acc = evaluate_model(part2, intermediate_output_test, y_test)  # Evaluate part2 with intermediate output
    print(f'Test accuracy (parallel training): {test_acc}')

    return test_acc

if __name__ == "__main__":
    # Measure time for the entire process
    start_time = datetime.datetime.now()

    print("Training parts in parallel and evaluating combined model...")
    parallel_model_acc = train_and_evaluate_parallel_parts()  # Train parts in parallel and evaluate the model

    # Measure end time for the entire process
    end_time = datetime.datetime.now()
    # Calculate total time for the entire process
    total_time = (end_time - start_time).total_seconds()
    print(f"Total time for the entire process: {total_time:.2f} seconds")

    # Print the final accuracy
    print(f"Parallel training - Test accuracy: {parallel_model_acc}")


Training parts in parallel and evaluating combined model...


Process Process-5:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-f51f773ebbeb>", line 58, in train_part2
    event.wait()
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 349, in wait
    self._cond.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 261, in wait
    return self._wait_semaphore.acquire(True, timeout)
KeyboardInterrupt


KeyboardInterrupt: 

In [1]:
import multiprocessing
import queue
import time
from PIL import Image
import os
import numpy as np
import tensorflow as tf

def download_mnist_images():
    # Load the MNIST dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    image_dir = 'mnist_images'
    os.makedirs(image_dir, exist_ok=True)
    image_paths = []

    # Save some images from the dataset
    for i in range(6):
        image_path = os.path.join(image_dir, f'image_{i}.jpg')
        img = Image.fromarray(x_train[i])
        img.save(image_path)
        image_paths.append(image_path)

    return image_paths

def worker(work_queue, result_queue):
    while not work_queue.empty():
        try:
            image_path = work_queue.get_nowait()
            convert_to_grayscale(image_path)
            result_queue.put(f"Processed {image_path}")
        except queue.Empty:
            break

def convert_to_grayscale(image_path):
    img = Image.open(image_path).convert('L')  # Convert image to grayscale
    grayscale_path = f"grayscale_{os.path.basename(image_path)}"
    img.save(grayscale_path)
    time.sleep(1)  # Simulate processing time

if __name__ == "__main__":
    image_paths = download_mnist_images()
    work_queue = multiprocessing.Queue()
    result_queue = multiprocessing.Queue()

    for image_path in image_paths:
        work_queue.put(image_path)

    workers = []
    for i in range(3):  # Creating 3 workers
        process = multiprocessing.Process(target=worker, args=(work_queue, result_queue))
        process.start()
        workers.append(process)

    for worker in workers:
        worker.join()

    while not result_queue.empty():
        print(result_queue.get())


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Processed mnist_images/image_0.jpg
Processed mnist_images/image_1.jpg
Processed mnist_images/image_2.jpg
Processed mnist_images/image_3.jpg
Processed mnist_images/image_4.jpg
Processed mnist_images/image_5.jpg


In [2]:
import multiprocessing
import queue
import time
from PIL import Image
import os
import numpy as np
import tensorflow as tf
import datetime

def download_mnist_images():
    # Load the MNIST dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    image_dir = 'mnist_images'
    os.makedirs(image_dir, exist_ok=True)
    image_paths = []

    # Save some images from the dataset
    for i in range(60):
        image_path = os.path.join(image_dir, f'image_{i}.jpg')
        img = Image.fromarray(x_train[i])
        img.save(image_path)
        image_paths.append(image_path)

    return image_paths

def worker(work_queue, result_queue, worker_id, metrics_queue):
    start_time = time.time()
    tasks_processed = 0
    while not work_queue.empty():
        try:
            image_path = work_queue.get_nowait()
            convert_to_grayscale(image_path)
            result_queue.put(f"Processed {image_path}")
            tasks_processed += 1
        except queue.Empty:
            break
    end_time = time.time()
    metrics_queue.put((worker_id, tasks_processed, start_time, end_time))

def convert_to_grayscale(image_path):
    img = Image.open(image_path).convert('L')  # Convert image to grayscale
    grayscale_path = f"grayscale_{os.path.basename(image_path)}"
    img.save(grayscale_path)
    time.sleep(1)  # Simulate processing time

if __name__ == "__main__":
    start_time = datetime.datetime.now()
    image_paths = download_mnist_images()
    work_queue = multiprocessing.Queue()
    result_queue = multiprocessing.Queue()
    metrics_queue = multiprocessing.Queue()

    for image_path in image_paths:
        work_queue.put(image_path)

    workers = []
    for i in range(3):  # Creating 3 workers
        process = multiprocessing.Process(target=worker, args=(work_queue, result_queue, i, metrics_queue))
        process.start()
        workers.append(process)

    for worker in workers:
        worker.join()

    end_time = datetime.datetime.now()
    total_time = (end_time - start_time).total_seconds()

    while not result_queue.empty():
        print(result_queue.get())

    metrics = []
    while not metrics_queue.empty():
        metrics.append(metrics_queue.get())

    # Display metrics
    for worker_id, tasks_processed, start, end in metrics:
        worker_time = end - start
        print(f"Worker {worker_id} processed {tasks_processed} tasks in {worker_time:.2f} seconds")

    print(f"Total time for processing: {total_time:.2f} seconds")


Processed mnist_images/image_0.jpg
Processed mnist_images/image_1.jpg
Processed mnist_images/image_2.jpg
Processed mnist_images/image_3.jpg
Processed mnist_images/image_4.jpg
Processed mnist_images/image_5.jpg
Processed mnist_images/image_6.jpg
Processed mnist_images/image_7.jpg
Processed mnist_images/image_8.jpg
Processed mnist_images/image_9.jpg
Processed mnist_images/image_10.jpg
Processed mnist_images/image_11.jpg
Processed mnist_images/image_12.jpg
Processed mnist_images/image_13.jpg
Processed mnist_images/image_14.jpg
Processed mnist_images/image_15.jpg
Processed mnist_images/image_16.jpg
Processed mnist_images/image_17.jpg
Processed mnist_images/image_18.jpg
Processed mnist_images/image_19.jpg
Processed mnist_images/image_20.jpg
Processed mnist_images/image_21.jpg
Processed mnist_images/image_22.jpg
Processed mnist_images/image_23.jpg
Processed mnist_images/image_24.jpg
Processed mnist_images/image_25.jpg
Processed mnist_images/image_26.jpg
Processed mnist_images/image_27.jpg
Pr

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import datetime

def download_mnist_images():
    # Load the MNIST dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
    x_test = x_test.reshape(-1, 784).astype('float32') / 255.0
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test)

def create_part1():
    inputs = tf.keras.Input(shape=(784,))
    x = layers.Dense(128, activation='relu')(inputs)
    outputs = layers.Dense(64, activation='relu')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_part2():
    inputs = tf.keras.Input(shape=(64,))
    outputs = layers.Dense(10, activation='softmax')(inputs)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_combined_model(part1, part2):
    inputs = tf.keras.Input(shape=(784,))
    x = part1(inputs)
    outputs = part2(x)
    model = tf.keras.Model(inputs, outputs)
    return model

if __name__ == "__main__":
    start_time = datetime.datetime.now()
    (x_train, y_train), (x_test, y_test) = download_mnist_images()

    # Create strategy for distributing training
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        part1 = create_part1()
        part2 = create_part2()
        combined_model = create_combined_model(part1, part2)

        # Compile the combined model
        combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Training the combined model
    batch_size = 128
    epochs = 10

    combined_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

    # Evaluate the combined model
    test_loss, test_acc = combined_model.evaluate(x_test, y_test)
    print(f'Test accuracy: {test_acc}')

    end_time = datetime.datetime.now()
    total_time = (end_time - start_time).total_seconds()
    print(f"Total time for processing: {total_time:.2f} seconds")


Epoch 1/10
469/469 [==============================] - 6s 9ms/step - loss: 0.3349 - accuracy: 0.9062
Epoch 2/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1361 - accuracy: 0.9610
Epoch 3/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0943 - accuracy: 0.9722
Epoch 4/10
469/469 [==============================] - 3s 5ms/step - loss: 0.0731 - accuracy: 0.9779
Epoch 5/10
469/469 [==============================] - 3s 6ms/step - loss: 0.0574 - accuracy: 0.9826
Epoch 6/10
469/469 [==============================] - 3s 7ms/step - loss: 0.0458 - accuracy: 0.9859
Epoch 7/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0366 - accuracy: 0.9888
Epoch 8/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0302 - accuracy: 0.9906
Epoch 9/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0243 - accuracy: 0.9925
Epoch 10/10
313/313 [==============================] - 2s 6ms/step - loss: 0.0829 - accuracy: 0.9769

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import datetime

def download_mnist_images():
    # Load the MNIST dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
    x_test = x_test.reshape(-1, 784).astype('float32') / 255.0
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test)

def create_part1():
    inputs = tf.keras.Input(shape=(784,))
    x = layers.Dense(128, activation='relu')(inputs)
    outputs = layers.Dense(64, activation='relu')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_part2():
    inputs = tf.keras.Input(shape=(64,))
    outputs = layers.Dense(10, activation='softmax')(inputs)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_combined_model(part1, part2):
    inputs = tf.keras.Input(shape=(784,))
    x = part1(inputs)
    outputs = part2(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def evaluate_model(model, x_test, y_test):
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return test_loss, test_acc

def train_and_evaluate_combined_model():
    (x_train, y_train), (x_test, y_test) = download_mnist_images()

    # Create strategy for distributing training
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        part1 = create_part1()
        part2 = create_part2()
        combined_model = create_combined_model(part1, part2)

        # Compile the combined model
        combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Training the combined model
    batch_size = 128
    epochs = 10

    epoch_times = []

    for epoch in range(epochs):
        epoch_start_time = datetime.datetime.now()
        combined_model.fit(x_train, y_train, batch_size=batch_size, epochs=1, verbose=1)
        epoch_end_time = datetime.datetime.now()
        epoch_time = (epoch_end_time - epoch_start_time).total_seconds()
        epoch_times.append(epoch_time)
        print(f'Epoch {epoch+1}/{epochs} - Time: {epoch_time:.2f} seconds')

    # Evaluate the combined model
    test_loss, test_acc = evaluate_model(combined_model, x_test, y_test)
    print(f'Test accuracy: {test_acc}')

    total_time = sum(epoch_times)
    print(f"Total training time: {total_time:.2f} seconds")

    return test_acc, total_time

if __name__ == "__main__":
    start_time = datetime.datetime.now()

    print("Training and evaluating combined model...")
    combined_model_acc, combined_model_time = train_and_evaluate_combined_model()

    end_time = datetime.datetime.now()
    total_time = (end_time - start_time).total_seconds()
    print(f"Total time for the entire process: {total_time:.2f} seconds")

    print(f"Combined model - Test accuracy: {combined_model_acc}, Training time: {combined_model_time:.2f} seconds")


Training and evaluating combined model...
469/469 [==============================] - 3s 5ms/step - loss: 0.3287 - accuracy: 0.9089
Epoch 1/10 - Time: 3.87 seconds
469/469 [==============================] - 2s 5ms/step - loss: 0.1335 - accuracy: 0.9609
Epoch 2/10 - Time: 3.24 seconds
469/469 [==============================] - 3s 6ms/step - loss: 0.0933 - accuracy: 0.9720
Epoch 3/10 - Time: 5.96 seconds
469/469 [==============================] - 2s 5ms/step - loss: 0.0715 - accuracy: 0.9786
Epoch 4/10 - Time: 2.84 seconds
469/469 [==============================] - 2s 5ms/step - loss: 0.0562 - accuracy: 0.9832
Epoch 5/10 - Time: 3.21 seconds
469/469 [==============================] - 3s 7ms/step - loss: 0.0449 - accuracy: 0.9863
Epoch 6/10 - Time: 5.83 seconds
469/469 [==============================] - 2s 5ms/step - loss: 0.0369 - accuracy: 0.9890
Epoch 7/10 - Time: 3.19 seconds
469/469 [==============================] - 2s 5ms/step - loss: 0.0304 - accuracy: 0.9908
Epoch 8/10 - Time: 3.2

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import datetime

def download_mnist_images():
    # Load the MNIST dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    # Reshape and normalize the training and testing images
    x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
    x_test = x_test.reshape(-1, 784).astype('float32') / 255.0
    # Convert labels to categorical (one-hot encoded) format
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test)

def create_part1():
    inputs = tf.keras.Input(shape=(784,))
    x = layers.Dense(128, activation='relu')(inputs)
    outputs = layers.Dense(64, activation='relu')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_part2():
    inputs = tf.keras.Input(shape=(64,))
    outputs = layers.Dense(10, activation='softmax')(inputs)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_combined_model(part1, part2):
    inputs = tf.keras.Input(shape=(784,))
    x = part1(inputs)
    outputs = part2(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def evaluate_model(model, x_test, y_test):
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return test_loss, test_acc

def train_and_evaluate_combined_model():
    # Load and preprocess the MNIST dataset
    (x_train, y_train), (x_test, y_test) = download_mnist_images()

    # Create strategy for distributing training
    strategy = tf.distribute.MirroredStrategy()  # Use MirroredStrategy for multi-GPU training

    with strategy.scope():
        # Create parts of the model and combine them
        part1 = create_part1()
        part2 = create_part2()
        combined_model = create_combined_model(part1, part2)

        # Compile the combined model
        combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Training parameters
    batch_size = 128
    epochs = 10

    epoch_times = []  # List to store the time taken for each epoch

    for epoch in range(epochs):
        # Record start time of the epoch
        epoch_start_time = datetime.datetime.now()
        # Train the combined model for one epoch
        combined_model.fit(x_train, y_train, batch_size=batch_size, epochs=1, verbose=1)
        # Record end time of the epoch
        epoch_end_time = datetime.datetime.now()
        # Calculate the duration of the epoch
        epoch_time = (epoch_end_time - epoch_start_time).total_seconds()
        epoch_times.append(epoch_time)  # Store the epoch duration
        print(f'Epoch {epoch+1}/{epochs} - Time: {epoch_time:.2f} seconds')

    # Evaluate the combined model on the test dataset
    test_loss, test_acc = evaluate_model(combined_model, x_test, y_test)
    print(f'Test accuracy: {test_acc}')

    # Calculate total training time
    total_time = sum(epoch_times)
    print(f"Total training time: {total_time:.2f} seconds")

    return test_acc, total_time

def train_and_evaluate_parallel_parts():
    # Load and preprocess the MNIST dataset
    (x_train, y_train), (x_test, y_test) = download_mnist_images()

    # Create strategy for distributing training
    strategy = tf.distribute.MirroredStrategy()  # Use MirroredStrategy for multi-GPU training

    with strategy.scope():
        # Create parts of the model
        part1 = create_part1()
        part2 = create_part2()

        # Compile the models separately
        part1.compile(optimizer='adam', loss='mse')  # Use MSE to learn good features
        part2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Training parameters
    batch_size = 128
    epochs = 10

    # Train part1 to generate good features
    part1.fit(x_train, x_train, batch_size=batch_size, epochs=epochs, verbose=1)

    # Generate intermediate outputs using part1
    intermediate_output = part1.predict(x_train)

    # Train part2 using the intermediate outputs
    part2.fit(intermediate_output, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

    # Combine part1 and part2 for evaluation
    combined_model = create_combined_model(part1, part2)

    # Evaluate the combined model on the test dataset
    intermediate_output_test = part1.predict(x_test)
    test_loss, test_acc = evaluate_model(part2, intermediate_output_test, y_test)
    print(f'Test accuracy (parallel training): {test_acc}')

    return test_acc

if __name__ == "__main__":
    # Measure time for the entire process
    start_time = datetime.datetime.now()

    print("Training and evaluating combined model...")
    combined_model_acc, combined_model_time = train_and_evaluate_combined_model()  # Train and evaluate the model

    print("Training parts in parallel and evaluating combined model...")
    parallel_model_acc = train_and_evaluate_parallel_parts()  # Train parts in parallel and evaluate the model

    # Measure end time for the entire process
    end_time = datetime.datetime.now()
    # Calculate total time for the entire process
    total_time = (end_time - start_time).total_seconds()
    print(f"Total time for the entire process: {total_time:.2f} seconds")

    # Print the final accuracy and training time for both approaches
    print(f"Combined model - Test accuracy: {combined_model_acc}, Training time: {combined_model_time:.2f} seconds")
    print(f"Parallel training - Test accuracy: {parallel_model_acc}")


Training and evaluating combined model...
469/469 [==============================] - 3s 5ms/step - loss: 0.3352 - accuracy: 0.9050
Epoch 1/10 - Time: 6.38 seconds
469/469 [==============================] - 2s 5ms/step - loss: 0.1351 - accuracy: 0.9610
Epoch 2/10 - Time: 3.18 seconds
469/469 [==============================] - 3s 7ms/step - loss: 0.0937 - accuracy: 0.9728
Epoch 3/10 - Time: 5.94 seconds
469/469 [==============================] - 2s 5ms/step - loss: 0.0699 - accuracy: 0.9788
Epoch 4/10 - Time: 3.06 seconds
469/469 [==============================] - 2s 5ms/step - loss: 0.0565 - accuracy: 0.9821
Epoch 5/10 - Time: 3.27 seconds
469/469 [==============================] - 3s 7ms/step - loss: 0.0460 - accuracy: 0.9860
Epoch 6/10 - Time: 4.16 seconds
469/469 [==============================] - 2s 5ms/step - loss: 0.0364 - accuracy: 0.9893
Epoch 7/10 - Time: 3.22 seconds
469/469 [==============================] - 2s 5ms/step - loss: 0.0303 - accuracy: 0.9905
Epoch 8/10 - Time: 3.2

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
import numpy as np
import multiprocessing
import datetime

def download_cifar10_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test)

def create_part1():
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    outputs = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_part2():
    inputs = tf.keras.Input(shape=(16, 16, 64))
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    outputs = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_part3():
    inputs = tf.keras.Input(shape=(8, 8, 256))
    x = layers.Flatten()(inputs)
    x = layers.Dense(512, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_combined_model(part1, part2, part3):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = part1(inputs)
    x = part2(x)
    outputs = part3(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def evaluate_model(model, x_test, y_test):
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return test_loss, test_acc

def train_part1(x_train, y_train, epochs, batch_size, return_dict, event):
    part1 = create_part1()
    part1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    part1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)
    intermediate_output = part1.predict(x_train)
    return_dict['part1'] = part1
    return_dict['intermediate_output1'] = intermediate_output
    event.set()

def train_part2(y_train, epochs, batch_size, return_dict, event1, event2):
    event1.wait()
    intermediate_output1 = return_dict['intermediate_output1']
    part2 = create_part2()
    part2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    part2.fit(intermediate_output1, y_train, batch_size=batch_size, epochs=epochs, verbose=1)
    intermediate_output2 = part2.predict(intermediate_output1)
    return_dict['part2'] = part2
    return_dict['intermediate_output2'] = intermediate_output2
    event2.set()

def train_part3(y_train, epochs, batch_size, return_dict, event2):
    event2.wait()
    intermediate_output2 = return_dict['intermediate_output2']
    part3 = create_part3()
    part3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    part3.fit(intermediate_output2, y_train, batch_size=batch_size, epochs=epochs, verbose=1)
    return_dict['part3'] = part3

def train_and_evaluate_parallel_parts():
    (x_train, y_train), (x_test, y_test) = download_cifar10_data()

    epochs = 10
    batch_size = 128

    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    event1 = multiprocessing.Event()
    event2 = multiprocessing.Event()

    process1 = multiprocessing.Process(target=train_part1, args=(x_train, y_train, epochs, batch_size, return_dict, event1))
    process2 = multiprocessing.Process(target=train_part2, args=(y_train, epochs, batch_size, return_dict, event1, event2))
    process3 = multiprocessing.Process(target=train_part3, args=(y_train, epochs, batch_size, return_dict, event2))

    process1.start()
    process2.start()
    process3.start()

    process1.join()
    process2.join()
    process3.join()

    part1 = return_dict['part1']
    part2 = return_dict['part2']
    part3 = return_dict['part3']

    combined_model = create_combined_model(part1, part2, part3)

    intermediate_output_test = part1.predict(x_test)
    intermediate_output_test = part2.predict(intermediate_output_test)
    test_loss, test_acc = evaluate_model(combined_model, x_test, y_test)
    print(f'Test accuracy (parallel training): {test_acc}')

    return test_acc

if __name__ == "__main__":
    start_time = datetime.datetime.now()

    print("Training parts in parallel and evaluating combined model...")
    parallel_model_acc = train_and_evaluate_parallel_parts()

    end_time = datetime.datetime.now()
    total_time = (end_time - start_time).total_seconds()
    print(f"Total time for the entire process: {total_time:.2f} seconds")
    print(f"Parallel training - Test accuracy: {parallel_model_acc}")


Training parts in parallel and evaluating combined model...


Process Process-17:
Process Process-18:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-eedabde3ee80>", line 72, in train_part3
    event2.wait()
  File "<ipython-input-6-eedabde3ee80>", line 61, in train_part2
    event1.wait()
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 349, in wait
    self._cond.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 349, in wait
    self._cond.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 261, in wa

KeyboardInterrupt: 

In [5]:
import tensorflow as tf  # Import TensorFlow library for building and training the neural network
from tensorflow.keras import layers, models, datasets  # Import layers, models, and datasets from Keras
import numpy as np  # Import NumPy for numerical operations
import multiprocessing  # Import multiprocessing for parallel processing
import datetime  # Import datetime for timing
import logging  # Import logging for debug messages

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def download_cifar10_data():
    logging.info("Downloading CIFAR-10 data...")
    # Load the CIFAR-10 dataset
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    # Normalize the pixel values to be between 0 and 1
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    # Convert class labels to one-hot encoded format
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    logging.info("CIFAR-10 data downloaded and preprocessed.")
    return (x_train, y_train), (x_test, y_test)

def create_part1():
    logging.info("Creating Part 1 of the model...")
    # Define the first part of the model
    inputs = tf.keras.Input(shape=(32, 32, 3))  # Input layer for CIFAR-10 images
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)  # Convolutional layer
    x = layers.MaxPooling2D((2, 2))(x)  # Max pooling layer
    outputs = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)  # Convolutional layer
    model = tf.keras.Model(inputs, outputs)  # Create the model
    logging.info("Part 1 created.")
    return model

def create_part2():
    logging.info("Creating Part 2 of the model...")
    # Define the second part of the model
    inputs = tf.keras.Input(shape=(16, 16, 64))  # Input layer for intermediate output
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(inputs)  # Convolutional layer
    x = layers.MaxPooling2D((2, 2))(x)  # Max pooling layer
    outputs = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)  # Convolutional layer
    model = tf.keras.Model(inputs, outputs)  # Create the model
    logging.info("Part 2 created.")
    return model

def create_part3():
    logging.info("Creating Part 3 of the model...")
    # Define the third part of the model
    inputs = tf.keras.Input(shape=(8, 8, 256))  # Input layer for intermediate output
    x = layers.Flatten()(inputs)  # Flatten layer to convert 3D output to 1D
    x = layers.Dense(512, activation='relu')(x)  # Fully connected layer
    outputs = layers.Dense(10, activation='softmax')(x)  # Output layer for classification
    model = tf.keras.Model(inputs, outputs)  # Create the model
    logging.info("Part 3 created.")
    return model

def create_combined_model(part1, part2, part3):
    logging.info("Combining parts into a single model...")
    # Combine part1, part2, and part3 into a single model
    inputs = tf.keras.Input(shape=(32, 32, 3))  # Input layer for CIFAR-10 images
    x = part1(inputs)  # Pass input through part1
    x = part2(x)  # Pass output of part1 through part2
    outputs = part3(x)  # Pass output of part2 through part3
    model = tf.keras.Model(inputs, outputs)  # Create the combined model
    logging.info("Combined model created.")
    return model

def evaluate_model(model, x_test, y_test):
    logging.info("Evaluating the model...")
    # Evaluate the model on the test dataset
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)  # Evaluate and get the loss and accuracy
    logging.info(f"Model evaluation completed. Test accuracy: {test_acc}")
    return test_loss, test_acc

def train_part1(x_train, y_train, epochs, batch_size, return_dict, event):
    logging.info("Training Part 1...")
    # Train part1 of the model
    part1 = create_part1()  # Create part1 model
    part1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Compile the model
    part1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)  # Train the model
    intermediate_output = part1.predict(x_train)  # Get the intermediate output
    return_dict['part1'] = part1  # Store the trained part1 model in the shared dictionary
    return_dict['intermediate_output1'] = intermediate_output  # Store the intermediate output in the shared dictionary
    logging.info("Part 1 training completed.")
    event.set()  # Signal that part1 has finished training

def train_part2(y_train, epochs, batch_size, return_dict, event1, event2):
    logging.info("Waiting for Part 1 to finish...")
    # Wait for part1 to finish and the intermediate output to be ready
    event1.wait()
    logging.info("Training Part 2...")
    intermediate_output1 = return_dict['intermediate_output1']  # Retrieve intermediate output from part1
    part2 = create_part2()  # Create part2 model
    part2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Compile the model
    part2.fit(intermediate_output1, y_train, batch_size=batch_size, epochs=epochs, verbose=1)  # Train the model
    intermediate_output2 = part2.predict(intermediate_output1)  # Get the intermediate output
    return_dict['part2'] = part2  # Store the trained part2 model in the shared dictionary
    return_dict['intermediate_output2'] = intermediate_output2  # Store the intermediate output in the shared dictionary
    logging.info("Part 2 training completed.")
    event2.set()  # Signal that part2 has finished training

def train_part3(y_train, epochs, batch_size, return_dict, event2):
    logging.info("Waiting for Part 2 to finish...")
    # Wait for part2 to finish and the intermediate output to be ready
    event2.wait()
    logging.info("Training Part 3...")
    intermediate_output2 = return_dict['intermediate_output2']  # Retrieve intermediate output from part2
    part3 = create_part3()  # Create part3 model
    part3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Compile the model
    part3.fit(intermediate_output2, y_train, batch_size=batch_size, epochs=epochs, verbose=1)  # Train the model
    return_dict['part3'] = part3  # Store the trained part3 model in the shared dictionary
    logging.info("Part 3 training completed.")

def train_and_evaluate_parallel_parts():
    logging.info("Starting training and evaluation of parallel parts...")
    # Load and preprocess the CIFAR-10 dataset
    (x_train, y_train), (x_test, y_test) = download_cifar10_data()

    # Training parameters
    epochs = 10
    batch_size = 128

    # Create a manager for shared dictionary and events for synchronization
    manager = multiprocessing.Manager()
    return_dict = manager.dict()  # Shared dictionary to store results from processes
    event1 = multiprocessing.Event()  # Event to signal when part1 has finished training
    event2 = multiprocessing.Event()  # Event to signal when part2 has finished training

    # Create processes for training part1, part2, and part3
    process1 = multiprocessing.Process(target=train_part1, args=(x_train, y_train, epochs, batch_size, return_dict, event1))
    process2 = multiprocessing.Process(target=train_part2, args=(y_train, epochs, batch_size, return_dict, event1, event2))
    process3 = multiprocessing.Process(target=train_part3, args=(y_train, epochs, batch_size, return_dict, event2))

    # Start the training processes
    process1.start()  # Start training part1
    process2.start()  # Start training part2
    process3.start()  # Start training part3

    # Wait for all processes to complete
    process1.join()
    process2.join()
    process3.join()

    # Retrieve the trained parts from the shared dictionary
    part1 = return_dict['part1']
    part2 = return_dict['part2']
    part3 = return_dict['part3']

    # Combine part1, part2, and part3 for evaluation
    combined_model = create_combined_model(part1, part2, part3)

    # Evaluate the combined model on the test dataset
    intermediate_output_test = part1.predict(x_test)  # Get intermediate output for test data from part1
    intermediate_output_test = part2.predict(intermediate_output_test)  # Get intermediate output for test data from part2
    test_loss, test_acc = evaluate_model(combined_model, x_test, y_test)  # Evaluate the combined model with test data
    logging.info(f'Test accuracy (parallel training): {test_acc}')

    return test_acc

if __name__ == "__main__":
    # Measure time for the entire process
    start_time = datetime.datetime.now()

    logging.info("Training parts in parallel and evaluating combined model...")
    parallel_model_acc = train_and_evaluate_parallel_parts()  # Train parts in parallel and evaluate the model

    # Measure end time for the entire process
    end_time = datetime.datetime.now()
    # Calculate total time for the entire process
    total_time = (end_time - start_time).total_seconds()
    logging.info(f"Total time for the entire process: {total_time:.2f} seconds")

    # Print the final accuracy
    logging.info(f"Parallel training - Test accuracy: {parallel_model_acc}")


Process Process-13:
Process Process-14:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-a7a042cc427d>", line 105, in train_part3
    event2.wait()
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 349, in wait
    self._cond.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 261, in wait
    return self._wait_semaphore.acquire(True, timeout)
  File "<ipython-input-5-a7a042cc427d>", line 90, in train_part2
    event1.wait()
  File "/usr/lib/python3.10/multiprocessing/synch

KeyboardInterrupt: 